__Задача.__  
Многоугольник задан координатами вершин.
Диагонали разбивают его на некоторое количество новых многоугольников.
Выписать все эти многоугольники.

Посмотреть этот файл можно
[здесь](https://github.com/raxal/edu-courses/blob/master/docs/letsko-polygons-1.ipynb) или 
[здесь](https://nbviewer.jupyter.org/github/raxal/edu-courses/blob/master/docs/letsko-polygons-1.ipynb).

In [ ]:
class Point:
    """Точка пересечения даигоналей"""
    
    def __init__(self, coord):
        self.x = coord[0]
        self.y = coord[1]
        self.xy = tuple(coord)
        self.diags = []
        self.nums = {}  # словарь: диагональ -> позиция точки на ней
    
    def __str__(self):
        return str(self.xy)
    
    def __repr__(self):
        return "Point({})".format(self.xy)

In [ ]:
data = [[-3,-4],[2,-4],[4,-1],[4,3],[0,7],[-5,3],[-5,-1]]
#data = [[-3,-4],[2,-4],[4,-1],[4,3],[0,7],[-5,-1],[-5,3]]  # не выпуклый
data = [[-3,-4],[2,-4],[4,-1],[4,3],[0,7],[-5,-1]]

In [ ]:
A = []
for pair in data:
    A.append(Point(pair))
N = len(A)

In [ ]:
pic = plot([])
for i in range(N):
    pic += plot(point(A[i].xy, color="red", size=30))
    pic += plot(text("$A[{}]$".format(i), (A[i].x + 0.3, A[i].y), color="red"))
for i in range(N):
    pic += plot(line([A[i].xy, A[(i+1)%N].xy]))
for i in range(N-2):
    for j in range(i+2, N):
        if (i,j) == (0,N-1):
            continue
        pic += line([A[i].xy, A[j].xy], thickness=0.3, color="green")
pic.show()

In [ ]:
import sympy.geometry as geom
x, y = var('x, y')
for i in range(N):
    line_sympy = geom.Line(A[i].xy, A[(i+1)%N].xy)
    eq = line_sympy.equation()._sage_()
    sg = sign(eq(x = A[(i+2)%N].x, y = A[(i+2)%N].y))
    for j in range(1, N-3):
        sg_test = sign(eq(x = A[(i+2+j)%N].x, y = A[(i+2+j)%N].y))
        assert sg_test == sg, "Многоугольник не выпуклый"
del geom
print("Многоугольник выпуклый")

In [ ]:
class Diag:
    """Диагональ"""
    
    def __init__(self, pair):
        self.name = tuple(pair)
        i,j = pair
        
        self.A = A[i]
        self.B = A[j]
        
        import sympy.geometry as geom
        x, y = var('x, y')
        line_sympy = geom.Line(A[i].xy, A[j].xy)
        self.eq = line_sympy.equation()._sage_()
        
        self.points = []
    
    def __str__(self):
        return str(self.name)
    
    def __repr__(self):
        return "Diag{}".format(self.name)

diags_list_names = []  # список всех диагоналей по "именам"
for i in range(N-2):
    for j in range(i+2, N):
        if (i,j) == (0,N-1):
            continue
        diags_list_names.append((i,j))

diags_list = []        # список всех диагоналей по объектам
diags_by_name = {}     # словарь для доступа к диагоналям по точкам
for pair in diags_list_names:
    diag_obj = Diag(pair)
    diags_list.append(diag_obj)
    diags_by_name[pair] = diag_obj

for i in range(len(diags_list)):
    for j in range(i+1, len(diags_list)):
        diag1 = diags_list[i]
        diag2 = diags_list[j]
        if (diag1.name[0] in diag2.name) or (diag1.name[1] in diag2.name):
            continue
        sol = solve([diag1.eq, diag2.eq], [x,y], solution_dict=True)
        if not sol:
            continue
        coord = (sol[0][x], sol[0][y])
        # проверка: лежит ли пересечение внутри полигона
        x_range = [diag1.A.x, diag1.B.x]
        y_range = [diag1.A.y, diag1.B.y]
        x_range.sort()
        y_range.sort()
        if (x_range[0] <= coord[0] <= x_range[1]) and \
           (y_range[0] <= coord[1] <= y_range[1]):
            point_obj = Point(coord)
            point_obj.diags.extend([diag1.name, diag2.name])
            diag1.points.append(point_obj)
            diag2.points.append(point_obj)

for diag in diags_list:
    diag.points.sort(key=lambda point_obj: point_obj.x)

for diag in diags_list:
    n = 0
    for point_obj in diag.points:
        point_obj.nums[diag.name] = n
        n += 1

In [ ]:
def neighbors(point_obj):
    """Возвращает соседние точки пересечения на всех диагоналях"""
    
    neighbors = []
    for diag in point_obj.diags:
        n = point_obj.nums[diag]
        if n > 0:
            neighbors.append(diags_by_name[diag].points[n-1])
        if n < len(diags_by_name[diag].points) - 1:
            neighbors.append(diags_by_name[diag].points[n+1])
    return neighbors

def orientation(A, B, C):
    """Выдает ориентацию векторов BA и BC по координатам A, B, C"""
    return sign((A[0]-B[0])*(C[1]-B[1])-(C[0]-B[0])*(A[1]-B[1]))

def angle_cos(A, B, C):
    """Выдает cos(BA, BC) для координат точек A, B, C"""
    BA = vector(A) - vector(B)
    BC = vector(C) - vector(B)
    return (BA * BC)/(BA.norm() * BC.norm())

In [ ]:
polygons_set = set([])
for diag in diags_list:
    for start_point in diag.points:
        n = start_point.nums[diag.name]
        if n == len(diag.points) - 1:
            break
        cur_pt = diag.points[n+1]
        cur_diag = diag.name
        new_polygon = [start_point]
        while cur_pt.xy != start_point.xy:
            new_polygon.append(cur_pt)
            for pair in cur_pt.diags:
                if pair != cur_diag:
                    next_diag = pair
                    break
            for next_pt in neighbors(cur_pt):
                if cur_diag not in next_pt.diags and \
                    orientation(new_polygon[-2].xy, cur_pt.xy, next_pt.xy) == -1:
                    cur_pt = next_pt
                    cur_diag = next_diag
            if cur_pt.xy == new_polygon[-1].xy:
                break
        if len(new_polygon) == 2:
            continue
        new_polygon = frozenset(new_polygon)
        polygons_set.add(new_polygon)

In [ ]:
polygons_set

In [ ]:
poly = list(polygons_set)[0]
pic1 = pic
for pt in poly:
    pic1 += plot(point(pt.xy, color="blue", size=30))
pic1.show()
print("{}-угольник".format(len(poly)))